In [1]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
from google.colab.patches import cv2_imshow
import pandas as pd
import re, os, sys, cv2
import numpy as np
from google.colab import drive
!pip install pybboxes
import pybboxes as pbx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#Нужно указать путь к папке с excel файлами и папкой images на гуглдиске
pth = '/content/drive/MyDrive/IT/NVI Solutions'

In [3]:
def draw_box(df_cat):
	df_cat['file'] = pd.read_excel(gt_path)['file'].iloc[df_cat.index]
	for index, row in df_cat.iterrows():
			img = cv2.imread(os.path.join(pth, 'images', row['file']))	# Read the image 
			img = cv2.resize(img, (600, 480))	# Resize the image to be displayed on the screen
			img = frame_box(img,row['truth_bbox'],color=(0,255,0)) # Call the function to draw the first box	
			img = frame_box(img,row['pred_bbox'],color=(255,0,0)) # Call the function to draw the second box	
			iou = get_iou(row['truth_bbox'], row['pred_bbox'])	# Calling the function to return the IOU
			img = cv2.putText(img, 'IOU: {}'.format(iou), (int(row['truth_bbox'][0]*1000), int(row['truth_bbox'][1])*1000), cv2.FONT_HERSHEY_SIMPLEX , 1, 
								(255,0,0), 2, cv2.LINE_AA) 	# Draw the IOU on the image
			print("IOU OF THE BOXES IS: ", iou)	# Print the iou
			cv2_imshow(img)	# Show the image

In [4]:
def frame_box(img, bbox, color=(255,0,0)):
  yolo_normalized = bbox 
  H, W = 600, 480#img.shape[:2]
  box_voc = pbx.convert_bbox(yolo_normalized, from_type="yolo", to_type="voc", image_size=(W,H))
  print(box_voc)
  cv2.rectangle(img, pt1=(box_voc[0], box_voc[1]), pt2=(box_voc[2], box_voc[3]), color=color, thickness=2)
  return img

In [5]:
def get_iou(ground_truth, pred):
    if (ground_truth != ground_truth) | (pred != pred):
      return float('nan')
    else:
    # coordinates of the area of intersection.
      ix1 = np.maximum(ground_truth[0], pred[0])
      iy1 = np.maximum(ground_truth[1], pred[1])
      ix2 = np.minimum(ground_truth[2], pred[2])
      iy2 = np.minimum(ground_truth[3], pred[3])
    # Intersection height and width.
      i_height = np.maximum(iy2 - iy1 + 1, np.array(0.))
      i_width = np.maximum(ix2 - ix1 + 1, np.array(0.))
      area_of_intersection = i_height * i_width
    # Ground Truth dimensions.
      gt_height = ground_truth[3] - ground_truth[1] + 1
      gt_width = ground_truth[2] - ground_truth[0] + 1
    # Prediction dimensions.
      pd_height = pred[3] - pred[1] + 1
      pd_width = pred[2] - pred[0] + 1
      area_of_union = gt_height * gt_width + pd_height * pd_width - area_of_intersection
      iou = area_of_intersection / area_of_union
      return iou

In [6]:
def str_lst(st):
   if pd.isna(st):
    return float('nan')
   else:
    st = re.findall('\d+\.\d+', st)
    return list(map(float, st))

In [7]:
drive.mount('/content/drive')
gt_path = os.path.join(pth,'ground_truth.xlsx')
pred_path = os.path.join(pth,'predictions.xlsx')

Mounted at /content/drive


In [8]:
target = pd.read_excel(gt_path)['truth_label']
predictions = pd.read_excel(pred_path)['pred_label']
truth_bbox = pd.read_excel(gt_path)['truth_bbox'].apply(str_lst)
pred_bbox = pd.read_excel(pred_path)['pred_bbox'].apply(str_lst)

Построим матрицу ошибок:   
[TN - FP]   
[FN - TP] ,   
где TN - истинноотрицательные, FP - ложноположительные, FN - ложноотрицательные, TP - истинноположительные.


In [9]:
print(confusion_matrix(target, predictions))

[[15  7]
 [ 0 28]]


In [10]:
print(f'{confusion_matrix(target, predictions)[0][1]/sum(target)*100}%')

25.0%


Данная матрица показывает, что 20% всех положительных ответов - ложноположительные, что свидетельствует о её переобученности. Требуется оптимизация и подбор параметров.

Для определения качества бинарной классификации в результатах модели распознования объектов, используем метрики:   
recall = TP / (TP + FN)(доля правильно определённых положительных ответов из всех истинно положительных),   
precision = TP / (TP + FP)(доля правильно определённых положительных ответов из всех положительных ответов модели).   

Для учёта обеих указанных метрик, посчитаем f1_score - среднее гармоническое полноты и точности f1_score = 2 * precision * recall / (precision + recall)

In [11]:
print('Метрика recall:')
print(recall_score(target, predictions))
print('Метрика precision:')
print(precision_score(target, predictions))
print('Метрика f1_score:')
print(f1_score(target, predictions))

Метрика recall:
1.0
Метрика precision:
0.8
Метрика f1_score:
0.888888888888889


Посчитаем метрику Intersection over Union (IoU), по площади пересечения обучающей разметки изображений и разметки, выполненной моделью.


In [12]:
df_bbox = pd.DataFrame({'truth_bbox':truth_bbox, 'pred_bbox':pred_bbox})
df_bbox['iou'] = df_bbox.apply(lambda x: get_iou(x['truth_bbox'], x['pred_bbox']), axis=1)
print('Cтатистическая сводка столбца iou:')
print(df_bbox['iou'].describe(), '\n')
print('Посмотрим какие изображения распознались моделью хуже всего IOU <= 0.731112')
draw_box(df_bbox[df_bbox['iou'] <= 0.731112])
print('Метрики IOU ложноположительных ответов модели отметили значением \'0\':')
print(df_bbox[df_bbox['iou']==0], '\n')
print('Медианное значение метрики IOU:')
print(df_bbox[['iou']].median(), '\n')
print('Среднее значение метрики IOU:')
print(df_bbox[['iou']].mean())

Output hidden; open in https://colab.research.google.com to view.

In [14]:
print('Посмотрим какие изображения распознались моделью лучше всего IOU > 0.731112')
draw_box(df_bbox[df_bbox['iou'] >= 0.731112])

Output hidden; open in https://colab.research.google.com to view.

Исходя из данного исследования, слдеует улучшить подбор параметров модели, чтобы уменьшить её переобученность, нужно увеличить выборку - датасет размеченный вручную с прецезионной разметкой. Затем повторить исследование.